In [1]:
from google.colab import drive
drive.mount('/content/drive') #replace with drive.mount('/content/drive/', force_remount=True) if the drive has changed since last mount in order to force the remount
%cd /content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions


In [2]:
!apt-get update
!apt-get install -y xvfb ffmpeg libsm6 libxext6
!pip install git+https://github.com/openai/CLIP.git
!pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
!pip install open3d pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (135 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [2]:
#stuff for importing PointCLIP
%cd PointCLIP
!pip install -r requirements.txt
%cd Dassl3D/
!python setup.py develop
%cd ..
%cd ..

/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/PointCLIP
/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/PointCLIP/Dassl3D
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
       

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/PointCLIP/trainers')
from zeroshot import PointCLIP_ZS

#Available clip models
import clip
clip.available_models()


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as F

from itertools import permutations, product
from Normalization import MeshNormalizer
from render import Renderer
from mesh import Mesh
from pathlib import Path
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh
import open3d as o3d
from pyvirtualdisplay import Display

width = 256
depth = 4       #default is 4
out_dim = 2
input_dim = 3
n_augs = 1      #default is 1

class NeuralHighlighter(nn.Module):
    def __init__(self):
        super(NeuralHighlighter, self).__init__()
        input_size = 3 #Dimension of the vertex
        output_size = 2 #Dimension of the output
                        #for the standard highlighter task there are only 2 classes: target region and not target region.
                        #we use the element of the output vector corresponding to the probability of belonging to the target
                        #region as the highlight probability described in the main paper.
        layers = []

        #See Appendix B (page 13)
        #first linear layer followed by ReLU and LayerNorm
        layers.append(nn.Linear(input_dim, width))
        layers.append(nn.ReLU())
        layers.append(nn.LayerNorm([width]))
        #other [depth] linear layers followed by ReLU and LayerNorm
        # -> changing the depth hyperparameter results in a deeper/shallower net
        # -> total depth (in terms of modules[Linear+ReLU+LayerNorm]) = [depth] + 1
        for i in range(depth):
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm([width]))
        #last linear layer followed by softmax in order to output probability-like values
        layers.append(nn.Linear(width, out_dim))
        layers.append(nn.Softmax(dim=1))

        self.mlp = nn.ModuleList(layers)
        self.model = self.mlp
        print(self.mlp)

    def forward(self, x):
        for layer in self.model:
            x = layer(x)
        return x

def get_clip_model(clipmodel):
    model, preprocess = clip.load(clipmodel, device=device)
    return model, preprocess

# ================== HELPER FUNCTIONS =============================
def save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=5,
                                                                        show=False,
                                                                        center_azim=0,
                                                                        center_elev=0,
                                                                        std=1,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        mesh.export(os.path.join(log_dir, f"{name}.ply"), extension="ply", color=final_color)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')

#TODO: fix the generation of the point cloud subsequently
#      now the point cloud generation is possible only by executing the PC_rendering.ipynb

def save_point_cloud_results(vertices, log_dir, name, mlp):
    '''
        #now i load the highlighted mesh and transpose it back to the point cloud
        display = Display(visible=0, size=(1400, 900))
        display.start()
        mesh_o3d = o3d.io.read_triangle_mesh(os.path.join(log_dir, f"{name}.ply"))

        if not mesh_o3d.has_vertex_normals():
          mesh_o3d.compute_vertex_normals()

        point_cloud = mesh_o3d.sample_points_poisson_disk(number_of_points=5000)

        output_ply_file = os.path.join(log_dir, f"{name}_point_cloud.ply")
        o3d.io.write_point_cloud(output_ply_file, point_cloud)
    '''
    mlp.eval()
    with torch.no_grad():
      probs = mlp(vertices)

      highlight = torch.tensor([204, 255, 0]).to(device)
      gray = torch.tensor([180, 180, 180]).to(device)
      colors = torch.stack((highlight/255, gray/255)).to(device)

      point_colors = assign_colors(probs, colors, device)
      point_cloud = o3d.geometry.PointCloud()
      point_cloud.points = o3d.utility.Vector3dVector(vertices)
      point_cloud.colors = o3d.utility.Vector3dVector(colors)

      o3d.io.write_point_cloud(f"{name}_colored_point_cloud.ply", point_cloud)

      '''
        width_final_render, height_final_render = 1400, 900
        render_final_pc = o3d.visualization.rendering.OffscreenRenderer(width_final_render, height_final_render)
        material = o3d.visualization.rendering.MaterialRecord()
        material.shader = "defaultUnlit"
        render_final_pc.scene.add_geometry("point_cloud", point_cloud, material)

        zoom_out_factor = 0.5
        bounding_box = point_cloud.get_axis_aligned_bounding_box()
        center = bounding_box.get_center()
        extent = bounding_box.get_extent()
        render_final_pc.scene.camera.look_at(center, center + [0, 0, 1], [0, 1, 0])
        render_final_pc.scene.camera.set_projection(60 / zoom_out_factor, width_final_render / height_final_render, 0.1, 100.0,
                                      o3d.visualization.rendering.Camera.FovType.Horizontal)

        pc_img = render_final_pc.render_to_image()
        output_file = os.path.join(log_dir, f"{name}_final_render.jpg")
        o3d.io.write_image(output_file, pc_img)

        display.stop()
      '''

def clip_loss(rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if encoded_text.shape[0] > 1:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
        else:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)

    elif n_augs > 0:
        loss = 0.0
        for _ in range(n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if encoded_text.shape[0] > 1:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
    return loss

def pointClip_loss(point_cloud, point_cloud_colors, pointClipZS):


    logits = pointClipZS.model_inference(point_cloud, point_cloud_colors)

    #print(logits.shape) -> depends on the text prompt, we obtain a list of values.
    #print(logits)       -> scores (%) ? here we compute the mean.
    logits.requires_grad_()
    clip_score = torch.mean(logits)

    return clip_score

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))


Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.5.1


In [5]:
from torch.optim.lr_scheduler import MultiStepLR
from utils import assign_colors

# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

render_res = 224
learning_rate = 0.0001
n_iter = 2500
res = 224
obj_path = 'data/dog.obj'
#output_dir = './output/'
clip_model_name = 'ViT-B/16'

device = "cuda" if torch.cuda.is_available() else "cpu"

#Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

objbase, extension = os.path.splitext(os.path.basename(obj_path))

render = Renderer(dim=(render_res, render_res))

#---------- WARNING---------
#---------- now i read the mesh from the obj of the reconstructed mesh
#mesh = Mesh(obj_path)
#MeshNormalizer(mesh)()
#---------- WARNING---------

width_render, height_render = 1400, 900
zoom_out_factor = 0.4

#------------ MESH TO POINT CLOUD INIT---------------
from pyvirtualdisplay import Display
display = Display(visible=0, size=(width_render, height_render))
display.start()

#------- here we retrieve first the point cloud from the mesh (only for test purpose)
mesh_o3d = o3d.io.read_triangle_mesh(obj_path)
mesh_o3d.compute_vertex_normals()
pcd = mesh_o3d.sample_points_poisson_disk(10000)
pcd_points = np.asarray(pcd.points)
print("pcd_points shape")
print(pcd_points.shape)
#-------

#-------here we retrieve the point cloud from the ply file (extracted from AffordanceNet pkls)
#pcd = o3d.io.read_point_cloud('./pointClouds/validation/bottle/Bottle_point_cloud_0.ply')

render_pc = o3d.visualization.rendering.OffscreenRenderer(width_render, height_render)
material = o3d.visualization.rendering.MaterialRecord()
material.shader = "defaultUnlit"
render_pc.scene.add_geometry("point_cloud", pcd, material)

bounding_box = pcd.get_axis_aligned_bounding_box()
center = bounding_box.get_center()
extent = bounding_box.get_extent()
render_pc.scene.camera.look_at(center, center + [0, 0, 1], [0, 1, 0])
render_pc.scene.camera.set_projection(60 / zoom_out_factor, width_render / height_render, 0.1, 100.0,
                                      o3d.visualization.rendering.Camera.FovType.Horizontal)

img = render_pc.render_to_image()
output_file = "point_cloud_render.jpg"
o3d.io.write_image(output_file, img)

#------------ MESH TO POINT CLOUD END---------------

#then we approximate a mesh so we can still use the previously defined helper functions
#also the loss minimization should converge better

#------------ POINT CLOUD TO MESH INIT---------------

#--- TO USE WITH AFFORDANCENET PCs
#pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=25))
#radii = [0.05, 0.1, 0.2]

#rec_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, scale=1.1, linear_fit=False)
#rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

#---

radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

# Optionally, check if the reconstructed mesh is valid
if rec_mesh.is_empty():
    print("Mesh reconstruction failed.")
else:
    print("Mesh reconstruction successful!")

# Render the reconstructed mesh
render_mesh = o3d.visualization.rendering.OffscreenRenderer(width_render, height_render)
material_mesh = o3d.visualization.rendering.MaterialRecord()
material_mesh.shader = "defaultUnlit"
render_mesh.scene.add_geometry("reconstructed_mesh", rec_mesh, material_mesh)

# Set zoom-out factor and camera for reconstructed mesh
bounding_box = pcd.get_axis_aligned_bounding_box()
center = bounding_box.get_center()
render_mesh.scene.camera.look_at(center, center + [0, 0, 1], [0, 1, 0])
render_mesh.scene.camera.set_projection(60 / zoom_out_factor, width_render / height_render, 0.1, 100.0,
                                        o3d.visualization.rendering.Camera.FovType.Horizontal)

# Render and save the reconstructed mesh image
img_mesh = render_mesh.render_to_image()
output_file_mesh = "reconstructed_mesh_render.jpg"
o3d.io.write_image(output_file_mesh, img_mesh)

#Export the reconstructed mesh to an obj file (allows to reuse helper functions)
output_mesh_file = "reconstructed_mesh.obj"
o3d.io.write_triangle_mesh(output_mesh_file, rec_mesh)

#Import the mesh from the exported obj file
mesh = Mesh(output_mesh_file)
MeshNormalizer(mesh)()

display.stop()
#------------ POINT CLOUD TO MESH END---------------

# Initialize variables
background = torch.tensor((1., 1., 1.)).to(device)

#log_dir = output_dir

# CLIP and Augmentation Transforms
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
])

augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
])

# MLP Settings
mlp = NeuralHighlighter().to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

#introducing learning rate decay
#with the prompt horse/saddle the loss plateaus
#scheduler = StepLR(optim, step_size=300, gamma=0.1)

#scheduler = MultiStepLR(optim, milestones=[300, 1800], gamma=0.1)  # Decay at epoch 300 and 1800


# list of possible colors
rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
colors = torch.tensor(full_colors).to(device)

name = 'dogPointCLIP_d_{}_augs_{}'.format(depth, n_augs)
depth_maps_views = 6

# --- Prompt ---
# encode prompt with CLIP
clip_model, preprocess = get_clip_model(clip_model_name)
prompts = ['A 3D picture of a gray and white dog with highlighted hat'] #WARNING: with PointCLIP the prompt is hardcoded


for i, prompt in enumerate(prompts):

  pointClipZS = PointCLIP_ZS()
  output_dir = './output_{}_{}/'.format(name, i)
  Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)
  log_dir = output_dir

  #here we compute the text encoding only once
  #if we put it inside the loss, we repeat n_iter times the same computation
  with torch.no_grad():
    text_input = clip.tokenize([prompt]).to(device)
    encoded_text = clip_model.encode_text(text_input)
    encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

  #vertices = copy.deepcopy(mesh.vertices)
  #vertices = torch.tensor(np.asarray(rec_mesh.vertices), dtype=torch.float32, device=device) # Convert vertices to a PyTorch tensor
  vertices = torch.tensor(pcd_points, dtype=torch.float32, device=device) # Convert point cloud points to a PyTorch tensor
  n_views = 5

  losses = []

  # Optimization loop
  for i in tqdm(range(n_iter)):
    optim.zero_grad()

    # predict highlight probabilities
    pred_class = mlp(vertices)
    #print("pred_class shape")
    #print(pred_class.shape)

    #point cloud coloring
    point_colors = assign_colors(pred_class, colors, device)

    # color and render mesh
    #sampled_mesh = mesh
    #color_mesh(pred_class, sampled_mesh, colors)       # EDIT: utils.py/color_mesh edited in order to return colors used later for mesh export


    '''
    mesh_dir = os.path.join(log_dir, 'meshes')
    os.makedirs(mesh_dir, exist_ok=True)
    mesh_path = os.path.join(mesh_dir, f"mesh_{i}.ply")

    #here I take the colored mesh as input and I export it to a ply file
    mesh.export(mesh_path, extension="ply", color=colors)

    #then I obtain the point cloud from the exported mesh (check if we can lower the number of sampled points)
    mesh_o3d = o3d.io.read_triangle_mesh(mesh_path)
    mesh_o3d.compute_vertex_normals()
    pcd = mesh_o3d.sample_points_poisson_disk(10000)

    #----- now we generate the 2D depth maps which will be the inputs to the new loss function
    #first we extract points and colors
    points = torch.tensor(np.asarray(pcd.points), dtype=torch.float32).unsqueeze(0).cuda()  # Shape: [1, N, 3]
    colors = torch.tensor(np.asarray(pcd.colors), dtype=torch.float32).unsqueeze(0).cuda()  # Shape: [1, N, 3]

    #depth_maps = pc_views.get_img(points)
    color_tensors = []

    for view_idx in range(depth_maps_views):
      depth_map, color_tensor = points2depth_with_color(points, colors, render_res, render_res)
      color_tensors.append(color_tensor)

    #depth_map_tensor = torch.stack(depth_maps)  # [num_views, B, RESOLUTION, RESOLUTION]  -> gray scale maps
    color_map_tensor = torch.stack(color_tensors)  # [num_views, B, RESOLUTION, RESOLUTION, 3] -> rgb maps

    color_tensor_final = color_map_tensor.squeeze(1) #collapse batch dimension (=1) then make of size [n_views, C, H, W]

    #color_tensor_final = color_tensor_final.permute(0, 2, 3, 1) #[n_views, H, W, C]
    #print("Color tensor final permuted shape")
    #print(color_tensor_final.shape)

    '''

    '''
    rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                            show=False,
                                                            center_azim=0,
                                                            center_elev=0,
                                                            std=1,
                                                            return_views=True,
                                                             lighting=True,
                                                            background=background)


    # Calculate CLIP Loss
    loss = clip_loss(rendered_images, encoded_text, clip_transform, augment_transform, clip_model)
    '''

    #Calculate PointCLIP Loss
    loss = pointClip_loss(vertices, point_colors, pointClipZS)

    loss.backward(retain_graph=True)

    optim.step()

    #LR decay
    #scheduler.step()

    # update variables + record loss
    with torch.no_grad():
        losses.append(loss.item())

    # report results
    if i % 100 == 0:
        print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
        #save_renders(log_dir, i, rendered_images)
        #save_renders(log_dir, i, color_tensor_final)

        with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
            f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


  # save results
  #save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background)

  # save point cloud results
  save_point_cloud_results(vertices, log_dir, name, mlp)



  # Save prompts
  with open(os.path.join(output_dir, 'prompt.txt'), "w") as f:
    f.write(prompt)
    f.write("\n")
    f.write("initial learning rate:")
    f.write(str(learning_rate))
    f.write("\n")
    f.write("n_iter:")
    f.write(str(n_iter))
    f.write("\n")
    f.write("n_augs:")
    f.write(str(n_augs))
    f.write("\n")
    f.write("n_views:")
    f.write(str(n_views))
    f.write("\n")
    f.write("clip_model:")
    f.write(clip_model_name)
    f.write("\n")
    f.write("depth:")
    f.write(str(depth))


pcd_points shape
(10000, 3)
[Open3D INFO] EGL headless mode enabled.
Mesh reconstruction successful!
[Open3D WARNING] Write OBJ can not include triangle normals.
ModuleList(
  (0): Linear(in_features=3, out_features=256, bias=True)
  (1): ReLU()
  (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): ReLU()
  (11): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (12): Linear(in_features=256, out_features=256, bias=True)
  (13): ReLU()
  (14): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (15): Linear(in_features=256, out_features=2, bias=True)
  (16): Softmax(dim=1)
)
Loading CLIP (backbone: ViT-B/16)


  0%|          | 0/2500 [00:00<?, ?it/s]/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/PointCLIP/trainers/zeroshot.py:93: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  img = torch.nn.functional.upsample(img, size=(224, 224), mode='bilinear', align_corners=True)
  0%|          | 1/2500 [00:00<11:44,  3.55it/s]

torch.Size([1, 3072])
torch.Size([72, 3072])
Last 100 CLIP score: 57.96875
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])


  0%|          | 5/2500 [00:00<04:13,  9.84it/s]

torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])


...

100%|█████████▉| 2497/2500 [03:05<00:00, 13.55it/s]

torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])


100%|██████████| 2500/2500 [03:05<00:00, 13.47it/s]

torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])
torch.Size([1, 3072])
torch.Size([72, 3072])


RuntimeError: Unable to cast Python instance of type <class 'torch.Tensor'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [ ]:
torch.cuda.empty_cache()